In [1]:
from PIL import Image
import numpy as np

import cv2
import pathlib
from tqdm import tqdm
import pickle
from colorthief import ColorThief
from matplotlib.pyplot import imshow

In [2]:
root = pathlib.Path('/data1/Raw')
out = pathlib.Path('/data2/ResizedImages/Depth9')

In [3]:
all_images_paths = list(root.glob('**/*.jpg'))
all_image_data = []
wrong_sized_imgs = 0
count = 0
data_count = 0

print(len(all_images_paths))

446054


In [4]:
depth = 9
size = 4*(2**(depth-1))
FHD_WIDTH = size
FHD_HEIGHT = size
print(FHD_WIDTH, FHD_HEIGHT)

def fill_background(img, dom_color):
    width = img.shape[1]
    height = img.shape[0]
    
    hmargin = int((FHD_WIDTH - width) / 2)
    vmargin = int((FHD_HEIGHT - height) / 2)

    output = cv2.copyMakeBorder(
                 img, 
                 vmargin, 
                 vmargin, 
                 hmargin, 
                 hmargin, 
                 cv2.BORDER_CONSTANT, 
                 value=dom_color
              )

    return output

def resize_image(img, w, h):
    resized = cv2.resize(img, (int(w),int(h)))
    return resized

def fix_image_size(img, source):
    width = img.shape[1]
    height = img.shape[0]
    
    # too large/small
    ratio1 = FHD_WIDTH/width
    ratio2 = FHD_HEIGHT/height

    ratio = min(ratio1, ratio2)

    img = resize_image(img, ratio*width, ratio*height)
    
    if img.shape == (FHD_WIDTH, FHD_HEIGHT, 3):
        return img
    
    dom_color = ColorThief(str(source)).get_color(quality=1)
    img = fill_background(img, dom_color)
    
    if img.shape == (FHD_WIDTH, FHD_HEIGHT, 3):
        return img
    
    return resize_image(img, FHD_WIDTH, FHD_HEIGHT)

1024 1024


In [5]:
from multiprocessing import Pool
from tqdm import *

def worker(source):
    try:
        org_path = str(source).split('/')
        out_path = pathlib.Path(out, org_path[-3] + org_path[-2] + org_path[-1])

        if out_path.exists():
            img = cv2.imread(str(out_path))

            if img.shape == (FHD_WIDTH, FHD_HEIGHT, 3):
                return

        img = cv2.imread(str(source))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        if img.shape != (FHD_WIDTH, FHD_HEIGHT, 3):
            img = fix_image_size(img, source)

        out_path.parent.mkdir(parents=True, exist_ok=True)
        cv2.imwrite(str(out_path), img)
    except Exception as e:
        print('Error', source, e)

In [6]:
p = Pool(7)

pbar = tqdm(total=len(all_images_paths))
for i, _ in enumerate(p.imap_unordered(worker, all_images_paths)):
    pbar.update()

  0%|          | 1053/446054 [00:03<26:35, 278.89it/s] 

Error /data1/Raw/WallpaperPlay/initial-d-wallpapers/84744.jpg OpenCV(4.3.0) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'



  1%|          | 2506/446054 [00:09<24:55, 296.59it/s]

Error /data1/Raw/WallpaperPlay/blank-white-wallpapers/58065.jpg Empty pixels when quantize.


  5%|▍         | 21563/446054 [01:23<25:58, 272.44it/s]

...

 88%|████████▊ | 392807/446054 [47:26:01<13:41:23,  1.08it/s] /home/andrew/anaconda3/envs/gogh-get-money/lib/python3.7/site-packages/PIL/Image.py:2835: DecompressionBombWarning: Image size (107952000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning,
 89%|████████▊ | 395565/446054 [48:30:02<24:50:52,  1.77s/it] /home/andrew/anaconda3/envs/gogh-get-money/lib/python3.7/site-packages/PIL/Image.py:2835: DecompressionBombWarning: Image size (97118000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning,
 91%|█████████ | 404851/446054 [51:51:00<10:18:48,  1.11it/s] /home/andrew/anaconda3/envs/gogh-get-money/lib/python3.7/site-packages/PIL/Image.py:2835: DecompressionBombWarning: Image size (126786888 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning,
 92%|█████████▏| 411264/446054 [54:25:54<95:22:18,  9.87s/it] /home/and

Error /data1/Raw/WallpaperAccess/solid-color-iphone/1248737.jpg Empty pixels when quantize.


 96%|█████████▌| 428837/446054 [60:48:16<3:02:34,  1.57it/s] /home/andrew/anaconda3/envs/gogh-get-money/lib/python3.7/site-packages/PIL/Image.py:2835: DecompressionBombWarning: Image size (162000000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning,
 98%|█████████▊| 436289/446054 [63:35:39<44:44,  3.64it/s]   

Error /data1/Raw/WallpaperAccess/plain-white/2077889.jpg Empty pixels when quantize.


 98%|█████████▊| 436689/446054 [63:45:02<17:42:24,  6.81s/it]/home/andrew/anaconda3/envs/gogh-get-money/lib/python3.7/site-packages/PIL/Image.py:2835: DecompressionBombWarning: Image size (144400000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning,
100%|██████████| 446054/446054 [67:14:51<00:00,  1.30it/s]   

In [ ]:
p.close()
p.terminate()